In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int, Str
from aiida.work.workfunction import workfunction
from aiida.work.run import submit

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview
from collections import OrderedDict

from apps.surfaces.structure_browser import StructureBrowser
from apps.mcfitting.geooptwork import FTGeoOptWorkChain

# Geometry optimization

## Step 1: Select molecule

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        orig_structure = s
        try:
            atoms = struct_browser.results.value.get_ase()

            # Fixed Atoms
            first_slab_atom, last_slab_atom = np.argwhere(atoms.numbers == 79)[0, 0], len(atoms)

            slab = atoms.get_atomic_numbers()[first_slab_atom:last_slab_atom]
            slab_h = np.count_nonzero(slab==1)
            slab_three_rows = slab_h*3

            first_fixed_atom = len(atoms)-1568
            last_fixed_atom = len(atoms)

            text_fixed_atoms.value = '%d..%d'%(first_fixed_atom+1, last_fixed_atom)

            # Number of nodes
            int_machines.value = int(np.round(1.+len(atoms)/45.))
        except:
            print 'nope'

    update_view()
    
def update_view():
    with info_out:
        clear_output()

        inp_descr.value = ""

        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            inp_descr.value = orig_structure.description + " on slab"
            # atoms = orig_structure.get_ase()
            atoms = prepare_mol_on_slab(orig_structure.get_ase())

            # add new component
            viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
            viewer.add_unitcell()
            viewer.center()

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

In [ ]:
viewer = nglview.NGLWidget()
info_out = ipw.Output()
display(viewer, info_out)

## Step 2: Put it on the slab

In [ ]:
# dimensions of slab 

def prepare_mol_on_slab(mol):
    # determine cell size
    auz_top = 2.3302842402729535E+01
    dminau = 2.3
    
    cz = 40
    cx = 4.1637275703388632E+01
    cy = 4.1210215432584206E+01
    mol.cell = (cx,cy,cz)
    mol.pbc = (True,True,True)
    
    # position molecule a bit above gold slab
    mol.center()
    minz = np.amin(mol.positions[:,2])
    dz = (-minz + auz_top + dminau)
    mol.positions[:,2] += dz
    
    # template for gold slab
    au_slab = ase.io.read(filename='relaxed.xyz')
    au_slab = ase.build.sort(au_slab, tags=au_slab.get_positions()[:,2]*-1)
    
    mol_on_au = mol + au_slab
    return mol_on_au

@workfunction
def prepare_mol_on_slab_wf(orig_struct):
    orig_atoms = orig_struct.get_ase()
    new_atoms = prepare_mol_on_slab(orig_atoms)
    return StructureData(ase=new_atoms)

In [ ]:
def on_click_store(b):
    if not orig_structure:
        print("No structure selected.")
        return

    s = prepare_mol_on_slab_wf(orig_structure)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))
    try:
        print s.pk
    except:
        print 'no pk.'

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

## Step 3: Submit geometry optimization

In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    label = match[0]
    code = match[1]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..12 25 110',
                            description='Fixed Atoms',
                            style=style, layout=layout)

int_machines = ipw.IntText(value=1,
                           description='# Nodes',
                           style=style, layout=layout)

text_wavefunction = ipw.Text(placeholder='../asd.wfn',
                        description='Wavefunction',
                        style=style, layout=layout)


display(ipw.VBox([drop_computer, int_machines, text_fixed_atoms, text_wavefunction]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return

        cp2k_code = drop_computer.value
        struct = struct_browser.results.value
        fixed_atoms = Str(text_fixed_atoms.value)
        num_machines = Int(int_machines.value)
        wavefunction = Str(text_wavefunction.value)

        outputs = submit(FTGeoOptWorkChain,
                         cp2k_code=cp2k_code,
                         structure=struct,
                         fixed_atoms=fixed_atoms,
                         num_machines=num_machines,
                         wavefunction=wavefunction)
        print(outputs)



btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)